In [1]:
import os
from string import Template

In [2]:
SLURM_DIR = '../SLURMS/Figure1/'
os.makedirs(SLURM_DIR)

OSError: [Errno 17] File exists: '../SLURMS/Figure1/'

In [5]:
SLURM_TEMPLATE = Template('''#!/bin/bash
#SBATCH -p seas_dgx1
#SBATCH -n 1 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH --gres=gpu
#SBATCH --array=9-12
#SBATCH --mem=$memory
#SBATCH -t 3-00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=haehn@seas.harvard.edu
#SBATCH -o /n/home05/haehn/Projects/CP/SLURMS/Figure1/LOGS/%A_%a.out
#SBATCH -e /n/home05/haehn/Projects/CP/SLURMS/Figure1/LOGS/%A_%a.err

# Print this sub-job's task ID
echo "My SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID

source new-modules.sh

module load Anaconda/5.0.1-fasrc01

cd /n/home05/haehn/Projects/CP/EXP/

source activate CP

python run_regression.py $experiment $dataset $classifier $noise $SLURM_ARRAY_TASK_ID

# end
exit 0;
''')

In [6]:
Experiments = {
  'C.Figure1.position_common_scale': range(3),
  'C.Figure1.position_non_aligned_scale': range(4),
  'C.Figure1.length': range(4),
  'C.Figure1.direction': range(3),
  'C.Figure1.angle': range(3), 
  'C.Figure1.area': range(3),
  'C.Figure1.volume': range(3),
  'C.Figure1.curvature': range(4),
  'C.Figure1.shading': range(3)
}

Classifiers = {
  'MLP': '8000',
  'LeNet': '8000',
  'VGG19': '20000',
  'XCEPTION': '32000'
}

Noise = ['True']#, 'False']

for e in Experiments:
  Datasets = Experiments[e]
  for d in Datasets:
    for c in Classifiers:
      m = Classifiers[c]
      for n in Noise:
        
        new_slurm = SLURM_TEMPLATE.substitute(experiment=e, memory=m, dataset=str(d), classifier=c, noise=n, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
        with open(os.path.join(SLURM_DIR, e+'_'+str(d)+'_'+c+'_'+n+'.sbatch'), 'w') as f:
          f.write(new_slurm)
          

In [ ]:
for f in *_True.sbatch; do sbatch $f; done